# ML Model Management Demo Notebook
Use this notebook to experiment with the JFrog Artifactory ML Model Management feature and the Xray model scanning capabilities.

### ⚠️ Note: It is highly advised to clear your Hugging Face local model cache before running this demo.
## Set Up Hugging Face Environment

In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HUGGING_FACE_HUB_TOKEN=<IDENTITY-TOKEN>
# Replace the <PATH> placeholder with the path to your ML Model Management repository in Artifactory, found in the JFrog Platform SetMeUp.
%env HF_ENDPOINT=<PATH>

## Install Required PyPI Packages from JFrog Artifactory

In [ ]:
# Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up.
!pip3 install transformers numpy scikit-image pillow torch torchvision ipywidgets -i <ARTIFACTORY_PIP_REPOSITORY_URL>

## Import Required Packages

In [ ]:
import skimage
import numpy as np
from PIL import Image
from transformers import pipeline
from transformers.utils import logging
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
from PIL import ImageDraw
from PIL import ImageFont
import torch

# Ignore warnings
logging.set_verbosity_error()

---

# Run the Demo

## Block an Unlicensed Hugging Face Model using Xray
Prior to running this stage, create an Xray policy that blocks the download of unlicensed artifacts in you ML Model Management repository. For more information, see [Creating Xray Policies and Rules](https://jfrog.com/help/r/jfrog-security-documentation/creating-xray-policies-and-rules). 

In [ ]:
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError


try:
    snapshot_download(repo_id="finiteautomata/bertweet-base-sentiment-analysis", etag_timeout=1500000000)
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Block-Unknown' license policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

## Use A Hugging Face Model With Artifactory

### Load And Configure the OWL-ViT Transformer

In [ ]:
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError


try:
    snapshot_download(repo_id="google/owlvit-base-patch32", etag_timeout=1500000000)
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Block-Unknown' license policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

In [ ]:
checkpoint = "google/owlvit-base-patch32"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")
model = AutoModelForZeroShotObjectDetection.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

### Display the Base Image

In [ ]:
im = Image.open("./img/frog2.jpg")
display(im)

### Where Are the Frogs?

In [ ]:
text_queries = ["Frog", "Cat"]
inputs = processor(text=text_queries, images=im, return_tensors="pt")
draw = ImageDraw.Draw(im)

with torch.no_grad():
    outputs = model(**inputs)
    target_sizes = torch.tensor([im.size[::-1]])
    results = processor.post_process_object_detection(outputs, threshold=0.25, target_sizes=target_sizes)[0]

draw = ImageDraw.Draw(im)

scores = results["scores"].tolist()
labels = results["labels"].tolist()
boxes = results["boxes"].tolist()

for box, score, label in zip(boxes, scores, labels): 
    outline = "green"
    caption = text_queries[label]
    if label != 0: # Frog is the zero ordinal in lables
        outline = "red"
        caption = "Not a Frog!!!!"
        
    xmin, ymin, xmax, ymax = box
    font = ImageFont.truetype(r'fonts/OpenSans-VariableFont_wdth,wght.ttf', 60) 
    draw.rectangle((xmin, ymin, xmax, ymax), outline=outline, width=3)
    draw.text((xmin, ymin), f"{caption}", fill=outline, font = font)

display(im)
# Close image
im.close()